In [1]:
from fractalai.model import RandomDiscreteModel
from fractalai.environment import ExternalProcess, ParallelEnvironment, AtariEnvironment
from fractalai.swarm_wave import SwarmWave
from fractalai.dataset_creator import DataGenerator
import time

In [2]:
# SwarmWave?

# This algorithhm is made for solving discrete Markov decision processes when a perfect model is available

In this context, if you need to generate data to train a neural network, IO is now your bottleneck.

In [3]:
name = "MsPacman-ram-v0"
skip_frames = 0  # Number of frames to skip at the beginning
dt_mean = 5  # Apply the same action n times in average
dt_std = 3 # Repeat same action a variable number of times
min_dt = 2 # Minimum number of consecutive steps to be taken
n_samples = 500000  # Maximum number of samples allowed
reward_limit = 20000  # Stop the sampling when this score is reached
n_walkers = 60  # Maximum witdh of the tree containing al the trajectories
render_every = 1  # print statistics every n iterations.
balance = 2  # Balance exploration vs exploitation
save_data = True # Save the data generated



In [4]:
env = ParallelEnvironment(name=name,env_class=AtariEnvironment,
                          blocking=False, n_workers=8, min_dt=1)  # We will play an Atari game
model = RandomDiscreteModel(max_wakers=n_walkers,
                            n_actions=env.n_actions) # The Agent will take discrete actions at random

/usr/local/lib/python3.6/dist-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


In [5]:
data_env = AtariEnvironment("Pong-v0", min_dt=3)


/usr/local/lib/python3.6/dist-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


In [6]:
wave = SwarmWave(model=model, env=env, n_walkers=n_walkers, reward_limit=reward_limit, samples_limit=n_samples,
                  render_every=render_every, balance=balance, save_data=save_data,
                  dt_mean=dt_mean, dt_std=dt_std, accumulate_rewards=True, min_dt=min_dt,
                  prune_tree=True)


In [7]:
%%time
wave.run_swarm(print_swarm=True)

Environment: MsPacman-ram-v0 | Walkers: 60 | Deaths: 0 | data_size 100
Total samples: 243740 Progress: 100.50%
Reward: mean 19912.83 | Dispersion: 230.00 | max 20101.00 | min 19871.00 | std 44.03
Episode length: mean 4511.25 | Dispersion 26.00 | max 4527.00 | min 4501.00 | std 6.22
Dt: mean 4.87 | Dispersion: 11.00 | max 13.00 | min 2.00 | std 2.68
Status: Score limit reached.
Efficiency 0.44%
Generated 1068 Examples | 228.22 samples per example.

CPU times: user 6.27 s, sys: 345 ms, total: 6.61 s
Wall time: 31.2 s


In [8]:
# Increase sleep to make game render slower
wave.render_game(sleep=0.003)